Anova_tfidf_200features

In [1]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [2]:
import os 
import numpy as np 
import pandas as pd
from sklearn import preprocessing 
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier 

In [3]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/ANOVA_Tfidf/data.csv"
df = pd.read_csv(path)
df.head() 

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,Level 5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,Level 5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,Level 5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",Level 5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Level 2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [4]:
df.iloc[:,1].replace(['Level 1', 'Level 2', 'Level 3', 'Level 4', 'Level 5'],
                        [1, 2, 3, 4, 5], inplace=True)

In [5]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [6]:
#define bins
bins = [0,2,4,5]
#name of the group 
group_names = ['Low', 'Moderate', 'High']
df['Level '] = pd.cut(df["Level "],bins,labels = group_names) 

In [7]:
df.head(5)

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,High,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,High,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,High,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",High,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Low,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [8]:
df['Level '] = df['Level '].map({'Low': 1, 'Moderate': 2, 'High': 3}).astype(int)

In [9]:
df.head() 

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,3,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,3,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,3,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",3,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,1,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [10]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/ANOVA_Tfidf/tfidf_withstopwordchangelevels_200_features.csv"
x = pd.read_csv(path)
x.head() 

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,190,191,192,193,194,195,196,197,198,199
0,0,-0.086085,-0.106214,-0.086085,-0.20195,-0.086085,-0.040746,-0.113663,-0.048688,-0.069343,...,-0.086085,-0.082814,-0.041328,-0.086085,-0.086085,-0.086085,-0.339138,-0.086085,-0.086085,-0.086085
1,1,-0.086085,-0.106214,-0.086085,-0.20195,-0.086085,-0.040746,-0.113663,-0.048688,-0.069343,...,-0.086085,-0.082814,-0.041328,-0.086085,-0.086085,-0.086085,-0.339138,-0.086085,-0.086085,-0.086085
2,2,-0.086085,-0.106214,-0.086085,-0.20195,-0.086085,-0.040746,-0.113663,-0.048688,-0.069343,...,-0.086085,-0.082814,-0.041328,-0.086085,-0.086085,-0.086085,-0.339138,-0.086085,-0.086085,-0.086085
3,3,-0.086085,-0.106214,-0.086085,-0.20195,-0.086085,-0.040746,-0.113663,-0.048688,-0.069343,...,-0.086085,-0.082814,-0.041328,-0.086085,-0.086085,-0.086085,-0.339138,-0.086085,-0.086085,-0.086085
4,4,-0.086085,-0.106214,-0.086085,-0.20195,-0.086085,-0.040746,-0.113663,-0.048688,-0.069343,...,-0.086085,-0.082814,-0.041328,-0.086085,-0.086085,-0.086085,1.881556,-0.086085,-0.086085,-0.086085


In [11]:
x.drop(x.columns[[0]], axis=1, inplace=True)
x.head() 

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,-0.086085,-0.106214,-0.086085,-0.20195,-0.086085,-0.040746,-0.113663,-0.048688,-0.069343,-0.114812,...,-0.086085,-0.082814,-0.041328,-0.086085,-0.086085,-0.086085,-0.339138,-0.086085,-0.086085,-0.086085
1,-0.086085,-0.106214,-0.086085,-0.20195,-0.086085,-0.040746,-0.113663,-0.048688,-0.069343,-0.114812,...,-0.086085,-0.082814,-0.041328,-0.086085,-0.086085,-0.086085,-0.339138,-0.086085,-0.086085,-0.086085
2,-0.086085,-0.106214,-0.086085,-0.20195,-0.086085,-0.040746,-0.113663,-0.048688,-0.069343,-0.114812,...,-0.086085,-0.082814,-0.041328,-0.086085,-0.086085,-0.086085,-0.339138,-0.086085,-0.086085,-0.086085
3,-0.086085,-0.106214,-0.086085,-0.20195,-0.086085,-0.040746,-0.113663,-0.048688,-0.069343,-0.114812,...,-0.086085,-0.082814,-0.041328,-0.086085,-0.086085,-0.086085,-0.339138,-0.086085,-0.086085,-0.086085
4,-0.086085,-0.106214,-0.086085,-0.20195,-0.086085,-0.040746,-0.113663,-0.048688,-0.069343,-0.114812,...,-0.086085,-0.082814,-0.041328,-0.086085,-0.086085,-0.086085,1.881556,-0.086085,-0.086085,-0.086085


In [12]:
y=df['Level '] 

**Train Test Split**

In [13]:
# Train-Test-Split is performed

from sklearn.model_selection import train_test_split

# Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=42)
X_train.shape, X_test.shape

((1220, 200), (524, 200))

In [14]:
print(y_train.value_counts(normalize=True)*100)
print("\n")
print(y_test.value_counts(normalize=True)*100)

2    45.327869
1    29.590164
3    25.081967
Name: Level , dtype: float64


2    45.229008
1    29.961832
3    24.809160
Name: Level , dtype: float64


**Model Building**

In [15]:
def perceptron_classifier (X_train, y_train, X_test, y_test):
  p = Perceptron(random_state=150)
  p.fit(X_train, y_train)
  predictions_train = p.predict(X_train)
  predictions_test = p.predict(X_test)
  train_score = accuracy_score(predictions_train, y_train)
  #print("score on train data: ", train_score)
  test_score = f1_score(predictions_test, y_test, average='weighted' )
  print("Perceptron F1 score:", test_score)

In [16]:
from sklearn.linear_model import Perceptron
perceptron_classifier(X_train, y_train, X_test, y_test) 

Perceptron F1 score: 0.411804619644372


**Decision Tree**

In [17]:
clf_dt = DecisionTreeClassifier(max_depth=5, class_weight = "balanced")

In [18]:
clf_dt.fit(X_train,y_train)

DecisionTreeClassifier(class_weight='balanced', max_depth=5)

In [19]:
y_pred_train = clf_dt.predict(X_train)
y_pred_test = clf_dt.predict(X_test)

In [20]:
def classifcation_report_train_test(y_train, y_train_pred, y_test, y_test_pred):

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TRAIN DATA
            =========================================
            ''')
    print(classification_report(y_train, y_train_pred, digits=4))

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TEST DATA
            =========================================
            ''')
    print(classification_report(y_test, y_test_pred, digits=4))

In [21]:
scores = pd.DataFrame(columns=['Model', 'Train_Accuracy', 'Train_Recall', 'Train_Precision', 'Train_F1_Score', 
                               'Test_Accuracy', 'Test_Recall', 'Test_Precision', 'Test_F1_Score'])

def get_metrics(train_actual, train_predicted, test_actual, test_predicted, model_description, dataframe):

    train_accuracy  = accuracy_score(train_actual, train_predicted)
    train_recall    = recall_score(train_actual, train_predicted, average="weighted")
    train_precision = precision_score(train_actual, train_predicted, average="weighted")
    train_f1score   = f1_score(train_actual, train_predicted, average="weighted")
    
    test_accuracy   = accuracy_score(test_actual, test_predicted)
    test_recall     = recall_score(test_actual, test_predicted, average="weighted")
    test_precision  = precision_score(test_actual, test_predicted, average="weighted")
    test_f1score    = f1_score(test_actual, test_predicted, average="weighted")

    dataframe       = dataframe.append(pd.Series([model_description, 
                                                  train_accuracy, train_recall, train_precision, train_f1score,
                                                  test_accuracy, test_recall, test_precision, test_f1score],
                                                 index=scores.columns ), 
                                       ignore_index=True)

    return(dataframe)

In [22]:
scores = get_metrics(y_train, y_pred_train, y_test, y_pred_test, "DecisionTrees", scores)
scores

<ipython-input-21-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.34918,0.34918,0.628949,0.271957,0.29771,0.29771,0.436374,0.205141


**Random Forest**

In [23]:
clf_rf = RandomForestClassifier(class_weight = "balanced")

In [24]:
clf_rf.fit(X=X_train, y=y_train)

RandomForestClassifier(class_weight='balanced')

In [25]:
y_pred_train1 = clf_rf.predict(X_train)
y_pred_test1 = clf_rf.predict(X_test)

In [26]:
classifcation_report_train_test(y_train, y_pred_train1, y_test, y_pred_test1)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.9886    0.9584    0.9733       361
           2     0.9545    0.9873    0.9707       553
           3     0.9832    0.9575    0.9702       306

    accuracy                         0.9713      1220
   macro avg     0.9754    0.9678    0.9714      1220
weighted avg     0.9718    0.9713    0.9713      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3697    0.2803    0.3188       157
           2     0.4629    0.6582    0.5436       237
           3     0.2206    0.1154    0.1515       130

    accuracy                         0.4103       524
   macro avg     0.3511    0.3513    0.3380       524
weighted avg     0.3749    0.4103    0.3790       524



In [27]:
scores = get_metrics(y_train, y_pred_train1, y_test, y_pred_test1, "RandomForest", scores)
scores

<ipython-input-21-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.349180,0.349180,0.628949,0.271957,0.297710,0.297710,0.436374,0.205141
1,RandomForest,0.971311,0.971311,0.971806,0.971322,0.410305,0.410305,0.374878,0.378964


**AdaBoost**

In [ ]:
clf_adaboost =  AdaBoostClassifier()

In [ ]:
clf_adaboost.fit(X_train, y_train)

AdaBoostClassifier()

In [ ]:
y_pred_train2 = clf_adaboost.predict(X_train)
y_pred_test2 = clf_adaboost.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train2, y_test, y_pred_test2)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.6562    0.1745    0.2757       361
           2     0.4911    0.8987    0.6351       553
           3     0.5714    0.2092    0.3062       306

    accuracy                         0.5115      1220
   macro avg     0.5729    0.4275    0.4057      1220
weighted avg     0.5601    0.5115    0.4463      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3594    0.1465    0.2081       157
           2     0.4384    0.7806    0.5615       237
           3     0.2895    0.0846    0.1310       130

    accuracy                         0.4179       524
   macro avg     0.3624    0.3372    0.3002       524
weighted avg     0.3778    0.4179    0.3488       524



In [ ]:
scores = get_metrics(y_train, y_pred_train2, y_test, y_pred_test2, "Adaboost", scores)
scores

<ipython-input-21-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.472951,0.472951,0.682625,0.327366,0.448473,0.448473,0.427893,0.296461
1,RandomForest,0.971311,0.971311,0.972158,0.971323,0.417939,0.417939,0.383157,0.386565
2,Adaboost,0.511475,0.511475,0.560119,0.446287,0.417939,0.417939,0.377770,0.348793


**Gradient Boosting Classifier**

In [ ]:
clf_gbm = GradientBoostingClassifier()

In [ ]:
clf_gbm.fit(X=X_train, y=y_train)

GradientBoostingClassifier()

In [ ]:
y_pred_train3 = clf_gbm.predict(X_train)
y_pred_test3 = clf_gbm.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train3, y_test, y_pred_test3)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.9286    0.3601    0.5190       361
           2     0.5608    0.9928    0.7167       553
           3     0.9703    0.3203    0.4816       306

    accuracy                         0.6369      1220
   macro avg     0.8199    0.5577    0.5724      1220
weighted avg     0.7723    0.6369    0.5992      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3571    0.1592    0.2203       157
           2     0.4554    0.8186    0.5852       237
           3     0.2857    0.0615    0.1013       130

    accuracy                         0.4332       524
   macro avg     0.3661    0.3464    0.3022       524
weighted avg     0.3839    0.4332    0.3558       524



In [ ]:
scores = get_metrics(y_train, y_pred_train3, y_test, y_pred_test3, "GBM", scores)
scores

<ipython-input-21-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.472951,0.472951,0.682625,0.327366,0.448473,0.448473,0.427893,0.296461
1,RandomForest,0.971311,0.971311,0.972158,0.971323,0.417939,0.417939,0.383157,0.386565
2,Adaboost,0.511475,0.511475,0.560119,0.446287,0.417939,0.417939,0.377770,0.348793
3,GBM,0.636885,0.636885,0.772323,0.599219,0.433206,0.433206,0.383862,0.355807


**SVM Model**

In [28]:
linear_svm = SVC(kernel='linear', C=1, class_weight = "balanced")

In [29]:
linear_svm.fit(X=X_train, y= y_train)

SVC(C=1, class_weight='balanced', kernel='linear')

In [30]:
y_pred_train4 = linear_svm.predict(X_train)
y_pred_test4 = linear_svm.predict(X_test)

In [31]:
classifcation_report_train_test(y_train, y_pred_train4, y_test, y_pred_test4)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.4470    0.5374    0.4881       361
           2     0.6374    0.4069    0.4967       553
           3     0.3903    0.5523    0.4574       306

    accuracy                         0.4820      1220
   macro avg     0.4916    0.4989    0.4807      1220
weighted avg     0.5191    0.4820    0.4843      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3318    0.4459    0.3804       157
           2     0.4534    0.3080    0.3668       237
           3     0.2829    0.3308    0.3050       130

    accuracy                         0.3550       524
   macro avg     0.3560    0.3615    0.3507       524
weighted avg     0.3747    0.3550    0.3556       524



In [32]:
scores = get_metrics(y_train, y_pred_train4, y_test, y_pred_test4, "SVM", scores)
scores

<ipython-input-21-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.349180,0.349180,0.628949,0.271957,0.297710,0.297710,0.436374,0.205141
1,RandomForest,0.971311,0.971311,0.971806,0.971322,0.410305,0.410305,0.374878,0.378964
2,SVM,0.481967,0.481967,0.519081,0.484272,0.354962,0.354962,0.374659,0.355560


**KNN**

In [ ]:
knn = KNeighborsClassifier(n_neighbors=7)

In [ ]:
knn.fit(X=X_train, y=y_train)

KNeighborsClassifier(n_neighbors=7)

In [ ]:
y_pred_train5 = knn.predict(X_train)
y_pred_test5 = knn.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train5, y_test, y_pred_test5)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.5013    0.5374    0.5187       361
           2     0.5961    0.7125    0.6491       553
           3     0.4651    0.2614    0.3347       306

    accuracy                         0.5475      1220
   macro avg     0.5208    0.5038    0.5008      1220
weighted avg     0.5352    0.5475    0.5317      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3520    0.4013    0.3750       157
           2     0.4792    0.5823    0.5257       237
           3     0.3333    0.1462    0.2032       130

    accuracy                         0.4198       524
   macro avg     0.3882    0.3766    0.3680       524
weighted avg     0.4049    0.4198    0.4005       524



In [ ]:
scores = get_metrics(y_train, y_pred_train5, y_test, y_pred_test5, "KNN", scores)
scores

<ipython-input-21-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.472951,0.472951,0.682625,0.327366,0.448473,0.448473,0.427893,0.296461
1,RandomForest,0.971311,0.971311,0.972158,0.971323,0.417939,0.417939,0.383157,0.386565
2,Adaboost,0.511475,0.511475,0.560119,0.446287,0.417939,0.417939,0.377770,0.348793
3,GBM,0.636885,0.636885,0.772323,0.599219,0.433206,0.433206,0.383862,0.355807
4,SVM,0.518033,0.518033,0.573659,0.444810,0.423664,0.423664,0.341297,0.337309
5,KNN,0.547541,0.547541,0.535178,0.531666,0.419847,0.419847,0.404872,0.400547


**GNB**

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()

In [ ]:
nb.fit(X=X_train, y=y_train)

GaussianNB()

In [ ]:
y_pred_train6 = nb.predict(X_train)
y_pred_test6 = nb.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train6, y_test, y_pred_test6)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.7097    0.0609    0.1122       361
           2     0.7526    0.1320    0.2246       553
           3     0.2738    0.9771    0.4278       306

    accuracy                         0.3230      1220
   macro avg     0.5787    0.3900    0.2549      1220
weighted avg     0.6198    0.3230    0.2423      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.4118    0.0446    0.0805       157
           2     0.4510    0.0970    0.1597       237
           3     0.2478    0.8692    0.3857       130

    accuracy                         0.2729       524
   macro avg     0.3702    0.3370    0.2086       524
weighted avg     0.3888    0.2729    0.1920       524



In [ ]:
scores = get_metrics(y_train, y_pred_train6, y_test, y_pred_test6, "GNB", scores)
scores

<ipython-input-21-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.472951,0.472951,0.682625,0.327366,0.448473,0.448473,0.427893,0.296461
1,RandomForest,0.971311,0.971311,0.972158,0.971323,0.417939,0.417939,0.383157,0.386565
2,Adaboost,0.511475,0.511475,0.560119,0.446287,0.417939,0.417939,0.377770,0.348793
3,GBM,0.636885,0.636885,0.772323,0.599219,0.433206,0.433206,0.383862,0.355807
4,SVM,0.518033,0.518033,0.573659,0.444810,0.423664,0.423664,0.341297,0.337309
5,KNN,0.547541,0.547541,0.535178,0.531666,0.419847,0.419847,0.404872,0.400547
6,GNB,0.322951,0.322951,0.619799,0.242316,0.272901,0.272901,0.388825,0.192028
